# Tiny-TLS 1.3 Toy Implementation for COSI107a, Spring 2025, version 0.1

This document contains the implementation of a toy version of TLS 1.3, to be used as material for Brandeis' COSI107a course. The goal is to implement a minimalist version of TLS 1.3 that can communicate with a server using the protocol.

This is a work in progress, and it is expected that changes will be made to the protocol as we move forward


## 1. Import the necessary libraries

Since we're working with TLS in Python, it is helpful to use Python libraries that allow us to pack our data into binary form. TLS protocol specifies exact byte length and format, and we'll be doing a lot of conversion between numbers and bytes. We'll use the Python Struct library for this purpose

We'll also be importing the x25519 curve from the Cryptography library to generate the key used in our messages



In [436]:
import struct

from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf import hkdf
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

import socket
import time
from urllib.parse import urlparse


import os #for random nonce generation

# 2a. Define some helper functions

TLS represents all of its messages in byte form. As such, some elements (such as the content length) must be converted from integer to big edian, specifically with 2 bytes. We also need to be able to join the distinct elements in a message into a single byte slice


In [437]:
def u16_to_byte(x: int) -> bytes:
    return struct.pack('>H', x) # Use the struct package to pack a number into big-edian 2 bytes

def concatenate(*bufs: bytes) -> bytes: # Concatenate multiple byte slices into one singular byte slice
    return b''.join(bufs)

def print_bytes_as_hex(b: bytes) -> None: # This is to print the bytes as hex strings. Easier to double check this way
    hex_string = ' '.join([f'{x:02x}' for x in b])
    print(hex_string)

# 2b. Define our private key and public key

We can then generate the public key and private key using the x25519 curve from Cryptography


In [438]:
def key_pair() -> bytes:
    private_key = x25519.X25519PrivateKey.generate()
    public_key = private_key.public_key().public_bytes(
    encoding=serialization.Encoding.Raw,
    format=serialization.PublicFormat.Raw
    )
    return private_key, public_key

print(key_pair())


(<cryptography.hazmat.backends.openssl.x25519._X25519PrivateKey object at 0x7fe498164820>, b']h\xc3\xa9&m\xa7!\x85\x11\xeb\xb5\x9c]\x7f\x8e\xa2W\x96N\xfe\xc3\xd6vO\xae2\x9a\t\xe0[n')


Notice how each time we run the code, the public_key is different. This is intended behavior, as it prevents attackers from being able to predict our key pairs

# 3. The Extension Blueprint:

Extensions play a large part in shaping a TLS message. From negotiating supported key group to exchanging keys, all of these are achieved using extensions. Luckily for us, these different extensions have a common blueprint



In [439]:
def extension(id: int, content: bytes) -> bytes:
    return concatenate(
        u16_to_byte(id),                         #The ID of the extension. (e.g 0x0a = Supported Group)
        u16_to_byte(len(content)),               #Length of content
        content                                  #The actual content itself
    )

print_bytes_as_hex(extension(0x0a, bytes([0x00, 0x1d]))) #Example Supported Group extension


00 0a 00 02 00 1d


# 4. The ClientHello

With those building blocks, we can now write our first TLS message: The ClientHello. The ClientHello is always the first message to be sent in a TLS handshake, indicating that the client wants to connect with the server

The ClientHello has these components, in this order:
1. ProtocolVersion (Negotiate which version of TLS we're using)
2. Random Nonce (32 bit, for key generation)
3. Legacy Session ID (For our purposes, we won't be using sessions)
4. Cipher Suites (contains a suite of cipher - how to actually encrypt the key once we have it)
5. Legacy Compression Method (For TLS 1.3, this is null)
6. Extensions 

The code for it is as follows:

In [440]:
private_key, public_key = key_pair()

def client_hello() -> bytes:
    client_random = os.urandom(32)
    def key_share(pubkey: bytes) -> bytes:      # Encode our public key to be sent over the message
        return concatenate(
            u16_to_byte(len(pubkey) + 4),       # +4 represents the 4 extra byte before the pubkey (2 bytes for the x25519, 2 bytes for len of pubkey)
            u16_to_byte(0x1d),                  # 0x1d is the value for x25519 key
            u16_to_byte(len(pubkey)),           
            pubkey
        )
    
    def DNI(domain: str) -> bytes:
        return concatenate(
            u16_to_byte(len(bytes(domain,'utf-8')) + 3),
            bytes([0x00]),
            u16_to_byte(len(bytes(domain,'utf-8'))),
            bytes(domain, 'utf-8')
        )
    
    def extensions() -> bytes: #This intializes the extensions we need in our message
        return concatenate(
            extension(0x00, DNI('www.cloudflare.com')),
            extension(0x0a, bytes([0x00, 0x02, 0x00, 0x1d])), #Supported Group extensions. Currently only contains the x25519 curve
            extension(0x0d, bytes([0x00, 0x12, 0x04, 0x03, 0x08, 0x04, 0x04, 0x01, 0x05, 0x03, 0x08, 0x05, 0x05, 0x01, 0x08, 0x06, 0x06, 0x01, 0x02, 0x01])),
            extension(0x33, key_share(public_key)), #Key Share. Contains the public key generated from the x25519 curve
            extension(0x2b, bytes([0x02, 0x03, 0x04])) #TLS Version. This is how we negotiate TLS 1.3
        )
    
    def handshake() -> bytes: #This constitutes our actual ClientHello message
        return concatenate(
            bytes([0x03, 0x03]),                          # This value is for TLS 1.2. TLS 1.3 must disguise itself as TLS 1.2 to be received, after which it negotiates into TLS 1.3 through the TLS 1.3 extension
            client_random,                             # Random Nonce for key
            bytes([0x00]),                                # Session ID. Empty for our purposes
            bytes([0x00, 0x02, 0x13, 0x02]),              # Cipher Suite. We have a single cipher for our cipher suite (SHA384)
                                                          # I'm aware that there are 2 SHA384 ciphers: AES and CHACHA. I've included one here. Not sure
                                                          # if we need the other one or not
            bytes([0x01, 0x00]),                          # Compression Method. Empty for our purposes
            u16_to_byte(len(extensions())),
            extensions()
        )
    
    return concatenate(                                 #Include record layers for TLS 1.3 to complete message
        bytes([0x16, 0x03, 0x01]),
        u16_to_byte(len(handshake()) + 4),
        bytes([0x01,0x00]),
        u16_to_byte(len(handshake())),
        handshake()
    ), client_random

client_hello_msg, client_random = client_hello()
print_bytes_as_hex(client_hello_msg)
client_hello_no_record = client_hello_msg[5:] #This is to calculate the transcript hash


## To check the validity of the ClientHello, copy paste the ClientHello into the openSSL_debug.py script and run it. Server should give back a response

16 03 01 00 9b 01 00 00 97 03 03 62 ba a6 7d 49 bf df c4 eb 42 85 15 75 09 86 d3 31 cf 55 89 4b a8 d0 ed 11 58 79 94 3b 40 79 0b 00 00 02 13 02 01 00 00 6c 00 00 00 17 00 15 00 00 12 77 77 77 2e 63 6c 6f 75 64 66 6c 61 72 65 2e 63 6f 6d 00 0a 00 04 00 02 00 1d 00 0d 00 14 00 12 04 03 08 04 04 01 05 03 08 05 05 01 08 06 06 01 02 01 00 33 00 26 00 24 00 1d 00 20 e8 02 13 c4 1b 32 12 af af b8 69 d9 25 d2 e4 4f 4f 68 f3 a0 c2 df eb 0f f6 fc 78 a7 85 61 b5 25 00 2b 00 03 02 03 04


# 5 Sending the ClientHello:
Since we now have the ClientHello, the next step would be to send it over to a public domain and see if the server responds. For this part, we'll be using www.cloudbase.com, as it supports TLS 1.3, and the cipher suite matches with our clientHello.


In [441]:
server_hello_no_record = b""
encrypted_payload = b""

def parse_tls_record(data):
    #Parse a TLS record and return its components
    if len(data) < 5:  # Minimum TLS record length
        return None, None, None, data
    
    content_type = data[0] #Parse the record header to see which type of record this is.
    if (data[1] == 3) and (data[2] == 3):
        version = "1.2/1.3"
    length = int.from_bytes(data[3:5], byteorder='big')
    
    if len(data) < length + 5:
        return None, None, None, data
        
    if content_type == 0x17:
        payload = data[:5+length]  # Include the header for application data. We need the header for the decryption algorithm
        remaining_data = data[5+length:]
    else:
        payload = data[5:5+length]  # Original behavior for other types
        remaining_data = data[5+length:]
    
    content_type_names = { #Dictionary indicating the types of TLS records we may receive
        0x14: "Change Cipher Spec",
        0x15: "Alert",
        0x16: "Handshake",
        0x17: "Application Data"
    }
    
    type_name = content_type_names.get(content_type, f"Unknown ({content_type})")
    
    return type_name, version, payload, remaining_data

def parse_handshake_message(data): #This is exclusive for Handshake messages. We'll primarily use this to identify serverHellos.
    """Parse a TLS handshake message."""
    if len(data) < 4:
        return None, None, None
        
    msg_type = data[0]
    length = int.from_bytes(data[1:4], byteorder='big')
    
    handshake_types = {
        0x00: "Hello Request",
        0x01: "Client Hello",
        0x02: "Server Hello",
        0x0b: "Certificate",
        0x0c: "Server Key Exchange",
        0x0d: "Certificate Request",
        0x0e: "Server Hello Done",
        0x0f: "Certificate Verify",
        0x10: "Client Key Exchange",
        0x14: "Finished",
        0x08: "Encrypted Extensions",  # Added TLS 1.3 message types
        0x13: "Certificate Status"
    }
    
    type_name = handshake_types.get(msg_type, f"Unknown ({msg_type})")
    if len(data) >= 4+length:
        payload = data[:4+length]
    
    return type_name, length, payload

def format_bytes(data):
    """Format bytes as a hexadecimal string with ASCII representation."""
    hex_dump = ' '.join(f'{b:02x}' for b in data)
    return f"Hex: {hex_dump}"

def receive_all_data(sock, timeout=5):
    """Receive all data from the socket until timeout or connection close."""
    buffer = b""
    sock.settimeout(timeout)
    start_time = time.time()
    
    while True:
        try:
            chunk = sock.recv(16384)
            if not chunk:  # Connection closed by server
                break
            buffer += chunk
            
            # Check timeout
            if time.time() - start_time > timeout:
                break
                
        except socket.timeout:
            break
            
    return buffer
# Parse the URL to get the hostname
parsed_url = urlparse('https://www.cloudflare.com/')
host = parsed_url.hostname
port = 443

# Your existing client_hello
try:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.settimeout(5)
        print(f"Attempting to connect to {host}:{port}")
        s.connect((host, port))
        print("Connected to server.")
        
        bytes_sent = s.sendall(client_hello_msg)
        print(f"Sent ClientHello ({len(client_hello_msg)} bytes)")
        
        print("\nWaiting for server response...")
        try:
            buffer = receive_all_data(s)
            print(f"\nReceived {len(buffer)} new bytes")
            print(print_bytes_as_hex(buffer))


            
            # Process all complete TLS records in the buffer
            while buffer:
                record_type, version, payload, remaining = parse_tls_record(buffer)
                if record_type is None:  # Incomplete record
                    break
                    
                buffer = remaining
                print("\n" + "="*50)
                print(f"Record Type: {record_type}")
                print(f"TLS Version: {version}")
                
                if record_type == "Handshake":
                    msg_type, msg_length, msg_payload = parse_handshake_message(payload)
                    if msg_type:
                        print(f"Handshake Type: {msg_type}")
                        print(f"Message Length: {msg_length}")
                        if msg_payload:
                            print("\nMessage Payload:")
                            print(format_bytes(msg_payload))
                            server_hello_no_record = msg_payload
                else:
                    print("\nRecord Payload:")
                    if record_type == "Application Data":
                        encrypted_payload = payload
                    print(format_bytes(payload))
                
        except socket.timeout:
            print("Timeout while waiting for server response")
            
except ConnectionRefusedError:
    print("Connection refused. Is the OpenSSL server running?")
except socket.timeout:
    print("Connection attempt timed out")
except Exception as e:
    print(f"Error: {e}")

print("\nExtracted Data:")
if server_hello_no_record:
    print("\nServer Hello (without record header):")
    print(print_bytes_as_hex(server_hello_no_record))
if encrypted_payload:
    print("\nEncrypted Payload (with record header):")
    print(print_bytes_as_hex(encrypted_payload))

Attempting to connect to www.cloudflare.com:443
Connected to server.
Sent ClientHello (160 bytes)

Waiting for server response...

Received 2813 new bytes
16 03 03 00 5a 02 00 00 56 03 03 0c 62 b9 7a 38 7b 5d b9 76 5a 7a 57 5d 6f 91 98 da 73 3d cf 5d a6 20 6e 19 49 16 7c 00 60 d7 15 00 13 02 00 00 2e 00 33 00 24 00 1d 00 20 a6 cc 42 5a 9b 9f 74 9e 5b 00 b6 18 62 34 5c 4f 23 33 9e 5a 57 23 80 a5 cb d5 1b da 8e af 9e 02 00 2b 00 02 03 04 14 03 03 00 01 01 17 03 03 0a 93 46 49 f9 e0 1e 1e f2 ec df 21 b7 b8 e6 b2 92 4f 79 9b cd 76 1b 73 79 03 29 e7 ca 8f c5 0a c5 27 a0 de 61 f9 a8 66 4b 10 64 a5 6e cf 26 fb e3 d7 25 38 d0 72 03 bf 85 63 26 25 0c 2b 3f f3 88 37 5e c4 75 f7 4a aa f1 29 90 14 51 10 c8 b7 e4 d6 a1 5f 6a 2e c0 52 14 65 4a 0f a1 cb 14 92 31 cd 7c e3 e2 3d b0 dc 46 5d 0a 96 8e 2b a4 16 9a 5f e7 35 01 fa 98 b5 72 2f 78 cd b0 ac ed 5a aa 8c e6 5f da 98 1a e7 7e 9a f1 e4 44 78 ff 1e f2 ca e7 6a 91 4e 0f cd 4f ed 3e f9 80 c2 f8 8c 9b 35 82 eb bc d7 61 5c 55 37 77 a9 e3 4a ae 6e c0 3a

# 5. Parsing the ServerHello

Once the ClientHello is sent, the server responds with its own message. This message contains the ServerHello, the CipherChangeSpec, and the encrypted payload. Assuming that our ClientHello message is configured correctly, the server will respond with its chosen cipher suite and its own key. 

We are particularly interested in the server's key and server random, which we use to establish cryptographic parameters. We can then extract these properties and start the key calculation process

# 5a. Creating a Parser class 

The ServerHello message isn't always of the same size. Certain elements like the sessionID, as well as the content of extensions, may have variable length depending on the message itself. That's why we can't just extract the information based on indexes alone. We need a parser that would keep track of what element we're at and how many bytes we have to skip forward


In [442]:

class Parser:
    def __init__(self, data: bytes) -> None:
        self.data = data
        self.cursor = 0
    
    def skip(self, position: int) -> None:
        self.cursor += position

    def read(self, position: int) -> bytes:
        result = self.data[self.cursor : self.cursor + position]
        self.cursor += position
        return result 
    
    def read_uint8_prefixed(self) -> bytes:                           #Most extensions with variable lengths have bytes that denote their length. We can use this to skip forward appropriately 
        length = self.data[self.cursor]
        self.cursor += 1
        result = self.data[self.cursor : self.cursor + length]
        self.cursor += length
        return result

    def read_uint16_prefixed(self) -> bytes:
        length = int.from_bytes(self.data[self.cursor:self.cursor + 2], 'big') #Since some lengths are represented with 2 bytes, we need to convert them 
        self.cursor += 2
        result = self.data[self.cursor:self.cursor + length]
        self.cursor += length
        return result


# 5b. Parsing the ServerHello

With our Parser class, we can now parse the ServerHello to extract the ServerRandom and the public key. Keep in mind that we are only dealing with 1 cipher suite, 1 cryptographic and as such expects only 1 public key. A full version of TLS 1.3 will be much more complex

In [443]:
def server_hello_parser(msg: bytes):
    parser = Parser(msg)
    parser.skip(4)                                  #Skip HandShake Header
    parser.skip(2)                                  #Skip Server Version
    server_random = parser.read(32)                 #Get the serverRandom
    parser.read_uint8_prefixed()                    #Skip SessionID
    parser.skip(2)                                  #Skip Cipher Suite
    parser.skip(1)                                  #Skip Compression Method
    public_key = None
    extensions = parser.read_uint16_prefixed()
    extension_reader = Parser(extensions)
    while(extension_reader.cursor < len(extensions)):
        extension_type = extension_reader.read(2)
        extension_data = extension_reader.read_uint16_prefixed()
        if (extension_type == b'\x00\x33'):
            data = Parser(extension_data)
            data.skip(2)
            public_key = data.read_uint16_prefixed()
    return server_random, public_key


#Pre-generated server_hello_msg, used for debugging purposes
#server_hello_msg = b'\x02\x00\x00\x76\x03\x03\x70\x71\x72\x73\x74\x75\x76\x77\x78\x79\x7a\x7b\x7c\x7d\x7e\x7f\x80\x81\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x8b\x8c\x8d\x8e\x8f\x20\xe0\xe1\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xeb\xec\xed\xee\xef\xf0\xf1\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xfb\xfc\xfd\xfe\xff\x13\x02\x00\x00\x2e\x00\x2b\x00\x02\x03\x04\x00\x33\x00\x24\x00\x1d\x00\x20\x9f\xd7\xad\x6d\xcf\xf4\x29\x8d\xd3\xf9\x6d\x5b\x1b\x2a\xf9\x10\xa0\x53\x5b\x14\x88\xd7\xf8\xfa\xbb\x34\x9a\x98\x28\x80\xb6\x15'
#Run the script with the pre-generated server hello and compare with https://tls13.xargs.org/#server-handshake-keys-calc to see if the script is working
#Current serverHello generated from www.cloudbase.com. Right now I'm copy-pasting from the terminal when I run the debug.py script. Also parser is ignoring the record header.
server_random, public_key = server_hello_parser(server_hello_no_record)
print_bytes_as_hex(server_random)
print_bytes_as_hex(public_key)


           

        
        
        


0c 62 b9 7a 38 7b 5d b9 76 5a 7a 57 5d 6f 91 98 da 73 3d cf 5d a6 20 6e 19 49 16 7c 00 60 d7 15
a6 cc 42 5a 9b 9f 74 9e 5b 00 b6 18 62 34 5c 4f 23 33 9e 5a 57 23 80 a5 cb d5 1b da 8e af 9e 02


# 6. Key Derivation

Now that we have our private key and the server has sent back their public key, we can now encrypt our data to the server via key calculations. We can break down the key calculations into a few steps

# 6a. Transcript Hash

Transcript Hash refers to the hash of the ClientHello and the ServerHello messages. The idea is to associate these messages with the keys we are about to derive: These keys work with these messages. That way, even if an attacker somehow got hold of our keys, it would not work with whatever messages they try to send



In [444]:
def transcript_hash(client: bytes, server: bytes) -> bytes:
    digest = hashes.Hash(hashes.SHA384()) # We'll be using SHA384 to create the hash for our messages.
    digest.update(client)     # Note that both the ClientHello and ServerHello is used here
    digest.update(server)
    transcript_hash = digest.finalize()
    return transcript_hash

print_bytes_as_hex(client_hello_no_record)       #This is the clientHello message we generated from our code
print_bytes_as_hex(server_hello_no_record)       #Right now, this server_hello is pre-generated for testing purposes. We'll be parsing an actual server hello when the entire thing is ready

hello_hash = transcript_hash(client_hello_no_record, server_hello_no_record)
print_bytes_as_hex(hello_hash)

01 00 00 97 03 03 62 ba a6 7d 49 bf df c4 eb 42 85 15 75 09 86 d3 31 cf 55 89 4b a8 d0 ed 11 58 79 94 3b 40 79 0b 00 00 02 13 02 01 00 00 6c 00 00 00 17 00 15 00 00 12 77 77 77 2e 63 6c 6f 75 64 66 6c 61 72 65 2e 63 6f 6d 00 0a 00 04 00 02 00 1d 00 0d 00 14 00 12 04 03 08 04 04 01 05 03 08 05 05 01 08 06 06 01 02 01 00 33 00 26 00 24 00 1d 00 20 e8 02 13 c4 1b 32 12 af af b8 69 d9 25 d2 e4 4f 4f 68 f3 a0 c2 df eb 0f f6 fc 78 a7 85 61 b5 25 00 2b 00 03 02 03 04
02 00 00 56 03 03 0c 62 b9 7a 38 7b 5d b9 76 5a 7a 57 5d 6f 91 98 da 73 3d cf 5d a6 20 6e 19 49 16 7c 00 60 d7 15 00 13 02 00 00 2e 00 33 00 24 00 1d 00 20 a6 cc 42 5a 9b 9f 74 9e 5b 00 b6 18 62 34 5c 4f 23 33 9e 5a 57 23 80 a5 cb d5 1b da 8e af 9e 02 00 2b 00 02 03 04
14 fb a4 0c dd 52 20 02 00 11 9b 36 30 34 9f 0e d9 8e 55 5a fb 35 ac 19 a7 6e 02 ca 71 92 83 de e0 22 8a 18 ac ef e8 a2 30 8e d5 f2 6f 05 72 66


# 6a. Shared secret
The idea of the key exchange is that given the private key and the public key of the other party, both client and server can perform calculations to arrive at the same number. Essentially, client private x server public = server private x client public. This is called the shared secret. We'll perform the calculations on our side while the server does it on theirs.


In [445]:
# Our public key right now is raw bytes. Our private key, on the other hand, is a x25519PrivateKey object. The most convenient way to calculate the 
# shared secret is to turn the public key into a x25519PublicKey object and leverage the built in methods to generate the shared secret

server_public_key = x25519.X25519PublicKey.from_public_bytes(public_key) 
shared_secret = private_key.exchange(server_public_key)
print_bytes_as_hex(shared_secret)



41 aa 49 9e 66 8a a8 66 f5 31 f1 f9 48 f0 c3 26 f7 05 ee 16 eb 58 3e fc d9 f3 98 1b 97 55 db 73


# 6b. Early Secret, Derived Secret, Handshake Secret
We then go on to generate the early secret, the derived secret and finally the handshake secret. 

- Early secret is for 0-RTT data (essentially data sent with pre-shared keys). Since we're not using PSKs, this is a bunch of 0 bytes.
- Derived secret is mainly used as salt, mixed in with the early secret to get the handshake key. This way, even if an attacker has the early secret, it's hard to determine the handshake key

Each of these secrets have a "label" associated with them to identify their use, usually with the format "tls13 + ID of the component". 

* Note 1: We can pass in the label directly into the HKDF info parameter, or we can make a method that will append the TLS13 infront and add the necessary bytes for length, after which it is passed into the infor parameter. To avoid hardcoding, I went with the second options. If this example works, we can go back and try hardcoding the labels to see if it affects anything


In [446]:
## REMEMBER TO CHANGE IT BACK TO SHA256 AFTER TESTING. Algorithm goes back to SHA256, Length goes back to 32.


# UNCOMMENT THIS FOR TESTING. SHOULD WORK FOR SHA384 IMPLEMENTATIONS:

#shared_secret = bytes.fromhex('df4a291baa1eb7cfa6934b29b474baad2697e29f1f920dcc77c8a0a088447624')
#hello_hash = bytes.fromhex('e05f64fcd082bdb0dce473adf669c2769f257a1c75a51b7887468b5e0e7a7de4f4d34555112077f16e079019d5a845bd')

# expected result
# handshake secret: bdbbe8757494bef20de932598294ea65b5e6bf6dc5c02a960a2de2eaa9b07c929078d2caa09362.31c38d1725f179d299
# client secret: db89d2d6df0e84fed74a2288f8fd4d0959f790ff23946cdf4c26d85e51bebd42ae184501972f8d30c4a3e4a3693d0ef0
# server secret: 23323da031634b241dd37d61032b62a4f450584d1f7f47983ba2f7cc0cdcc39a68f481f2b019f9403a3051908a5d1622
# client handshake key: 1135b4826a9a70257e5a391ad93093dfd7c4214812f493b3e3daae1eb2b1ac69
# client handshake iv: 4256d2e0e88babdd05eb2f27
# server handshake key: 9f13575ce3f8cfc1df64a77ceaffe89700b492ad31b4fab01c4792be1b266b7f
# server handshake iv: 9563bc8b590f671f488d2da3


# TLS specs make use of 2 methods: HKDF-Extract and HKDF-Expand. HKDF-Extract can be found in hkdf.HKDF()_extract(). HKDF-Expand is hkdf.HKDFExpand()


#Define our own hkdf_expand_label. This is basically calling HKDF to derive secrets, with an added step of configuring the labels associated with the component.

def hkdf_expand_label(secret: bytes, label: str, context: bytes, length: int) -> bytes:
    # Construct the HkdfLabel as specified in TLS 1.3
    label = b"tls13 " + label.encode('utf-8')
    hkdf_label = (
        struct.pack("!H", length) +  # 2 bytes for length
        struct.pack("!B", len(label)) + label +  # 1 byte for label length + label
        struct.pack("!B", len(context)) + context  # 1 byte for context length + context
    )
    
    # Use HKDF-Expand
    encryption = hkdf.HKDFExpand(
        algorithm=hashes.SHA384(),
        length=length,
        info=hkdf_label,
    )
    
    return encryption.derive(secret)


early_secret = hkdf.HKDF(
    algorithm=hashes.SHA384(), #We negotiated SHA256 in our cipher suite
    length=48,
    salt=b'\x00',
    info=b'\x00'
)._extract(b'\x00' * 48) #The first input key is simply 00

empty_hash = hashes.Hash(hashes.SHA384())
empty_hash.update(b"")  # Empty string
empty_hash = empty_hash.finalize()

# We then derive another secret called the derived secret from the early secret
derived_secret = hkdf_expand_label(
    secret = early_secret,
    label="derived",
    context=empty_hash,
    length=48
)

# The derived secret is mixed into the early secret to get the handshake secret
handshake_secret = hkdf.HKDF(
    algorithm=hashes.SHA384(),
    length=48,
    salt=derived_secret,  # derived_secret is used as salt
    info=None
)._extract(shared_secret)

# 6c. Traffic Secrets and Key derivation

Now that we have our handshake secret, we can now move on to derive the traffic secrets, and subsequently the keys and IVs for both the client and the server

In [447]:
# Deriving the secrets and subsequent keys + IVs.
client_secret = hkdf_expand_label(
    secret = handshake_secret, 
    label = "c hs traffic", 
    context = hello_hash, 
    length = 48
)

server_secret = hkdf_expand_label(
    secret = handshake_secret,
    label = "s hs traffic",
    context = hello_hash,
    length = 48
)


client_handshake_key = hkdf_expand_label(
    secret = client_secret,
    label = "key",
    context = b"",
    length = 32
)

server_handshake_key = hkdf_expand_label(
    secret = server_secret,
    label = "key",
    context = b"",
    length = 32
)

client_handshake_iv = hkdf_expand_label(
    secret = client_secret,
    label = "iv",
    context = b"",
    length = 12
)

server_handshake_iv = hkdf_expand_label(
    secret = server_secret,
    label = "iv",
    context = b"",
    length = 12
)




print("handshake secret: " + handshake_secret.hex())
print("client secret: " + client_secret.hex())
print("server secret: " + server_secret.hex())
print("client handshake key: " + client_handshake_key.hex())
print("client handshake iv: "+ client_handshake_iv.hex())
print("server handshake key: " + server_handshake_key.hex())
print("server handshake iv: " + server_handshake_iv.hex())


handshake secret: b0a8055959e44ef9dc888bd4189114cf2d2a0b5a930b7df9962bd2c1a42dd9a3ea682e9f23e2eb5437faa115d5a0a5cd
client secret: 1b075c198b4233260dc8ed2c610b284bbc8bc64ab4b7350db78b62b3bc8ec5e729c8ebb7743f4d4caa082ff6f4a077df
server secret: 30d6929668747a27ff1c2430d75e2db3581a6c11b382991b7735e6d57923997a9d9b30fb23a349dc81c51c4e47af8919
client handshake key: d918de04f11b925e3a5d513cf9849fbbd4b1ce537319c72f512bf3c8825d9b04
client handshake iv: aaf86f6737d45b6d4881930c
server handshake key: ce901dc6af0df7fa79d1ed5e618846bff2a099c8e1dd4bab95ca3d5bd0020b40
server handshake iv: a3439a0b644e83e66264c971


## 7. Decryption

Now that we have the keys and IVs, the next step would be to decrypt the encrypted records. When sending the ClientHello, the server sends back a massive binary dump, including the ServerHello, the change cipher specs, and the actual payload, which is encrypted. We will use the keys and ivs generated to decrypt these records.

Normally, you'll see a bunch of records, each with its own 

In [448]:
def decrypt(key, iv, wrapper):
    # Split the wrapper into additional authenticated data (AAD) and ciphertext
    additional = wrapper[:5]
    ciphertext = wrapper[5:]
    
    # Initialize AES-GCM with the provided key
    aesgcm = AESGCM(key)
    
    # Decrypt the ciphertext
    try:
        plaintext = aesgcm.decrypt(iv, ciphertext, additional)
    except Exception as e:
        raise ValueError(f"Decryption failed: {e}")
    
    return plaintext

# Right now I'm doing a lot of copy pasting. I run the openSSL_debug script to retrieve the server's response, and then replace the serverHello, as well as the encrypted record under this line. Managed to decrypt the thing.

print_bytes_as_hex((decrypt(server_handshake_key, server_handshake_iv, encrypted_payload)))

08 00 00 06 00 04 00 00 00 00 0b 00 09 f1 00 00 09 ed 00 03 bd 30 82 03 b9 30 82 03 5f a0 03 02 01 02 02 10 6f 5c 9f ee 22 4c 96 2b 13 89 36 20 ce e5 f6 e4 30 0a 06 08 2a 86 48 ce 3d 04 03 02 30 3b 31 0b 30 09 06 03 55 04 06 13 02 55 53 31 1e 30 1c 06 03 55 04 0a 13 15 47 6f 6f 67 6c 65 20 54 72 75 73 74 20 53 65 72 76 69 63 65 73 31 0c 30 0a 06 03 55 04 03 13 03 57 45 31 30 1e 17 0d 32 35 30 31 31 34 32 30 35 38 34 32 5a 17 0d 32 35 30 34 31 34 32 31 35 38 33 39 5a 30 1d 31 1b 30 19 06 03 55 04 03 13 12 77 77 77 2e 63 6c 6f 75 64 66 6c 61 72 65 2e 63 6f 6d 30 59 30 13 06 07 2a 86 48 ce 3d 02 01 06 08 2a 86 48 ce 3d 03 01 07 03 42 00 04 9f ae 94 5d 92 32 76 30 52 f9 e0 0a 8b 22 e1 02 c9 0e dd 9d c8 50 7d cb fb 33 e7 8d b3 6d 46 3d cc ab 06 07 6b e4 60 7c e9 3d 49 fc 9c 0f c8 b9 96 09 a4 9e 5a 8c 8a 33 27 1f 1b 6d 25 c5 38 e9 a3 82 02 61 30 82 02 5d 30 0e 06 03 55 1d 0f 01 01 ff 04 04 03 02 07 80 30 13 06 03 55 1d 25 04 0c 30 0a 06 08 2b 06 01 05 05 07 03 01 30 0c 06 03 55 1d 13 01 01 f